# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [2]:
from rdflib import Namespace, URIRef, RDFS, Graph, Literal
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK, A, RDF
from buildingmotif.model_builder import TemplateBuilderContext

In [3]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
brick = Library.load(ontology_graph="Brick.ttl")

2025-07-07 12:14:40,065 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/quantitykind (Name: http://qudt.org/2.1/vocab/quantitykind). Check this is loaded into BuildingMOTIF
2025-07-07 12:14:40,066 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/qudt (Name: http://qudt.org/2.1/schema/shacl/qudt). Check this is loaded into BuildingMOTIF
2025-07-07 12:14:40,066 | root |  WARNING: An ontology could not resolve a dependency on https://w3id.org/rec/recimports (Name: https://w3id.org/rec/recimports). Check this is loaded into BuildingMOTIF
2025-07-07 12:14:40,067 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/unit (Name: http://qudt.org/2.1/vocab/unit). Check this is loaded into BuildingMOTIF
2025-07-07 12:14:40,067 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/dimensionvector (Name: http://qudt.org/2.1/vocab/dime

In [4]:
brick_tmpl = Library.load(directory='brick-templates')

In [5]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
# ahu = ctx['multiple-zone-ahu'](name='ahu1')
# # now create an instance of the water flow property
# flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
# flow_prop['name-owner'] = my_tank['out']
# # now create the sensor
# flow_sensor = ctx['sensor'](name='my_flow_sensor')
# flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
# flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [6]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs_per_ahu, reheat_vavs_per_ahu]
# Could add amt computer spaces_per_ahu to building if it serves example later

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(brick_tmpl)

In [7]:
# preparing both representations 

In [8]:
# bldg_config = {'ahus': 2, 'cv': 10, 'hv': 10}
# bldg_ctx_dict = {}
# for i in range(bldg_config['ahus']):
#     ahu_name = f'ahu_{i}'
#     # bldg_ctx_dict[name] = ctx['multiple-zone-ahu'](name=name)
#     ahu = ctx['multiple-zone-ahu'](name=ahu_name)
#     for cv in range(bldg_config['cv']):
#         cv_name = f'cvav_{i}_{cv}'
#         cvav = ctx['vav-cooling-only'](name=cv_name)
#         ctx['feeds'](name = ahu['name'], target = cvav['name'])
#     for hv in range(bldg_config['hv']):
#         hv_name = f'hvav_{i}_{hv}'
#         hvav = ctx['vav-with-reheat'](name=hv_name)
#         ctx['feeds'](name = ahu['name'], target = hvav['name'])

In [9]:
# TODO: MESSED UP CODE FLOW, NOW NEED TO RUN AGAIN FROM HERE
def get_wrapper_dict(ctx):
    wrapper_dict = {}
    for wrapper in ctx.wrappers:
        if wrapper.template.name not in wrapper_dict.keys():
            wrapper_dict[wrapper.template.name] = [wrapper]
        else:
            wrapper_dict[wrapper.template.name].append(wrapper)
    return wrapper_dict

In [10]:
wrapper_dict = get_wrapper_dict(ctx)
template_types = wrapper_dict.keys()
print(template_types)

dict_keys([])


In [11]:
TEMPLATE_PARAMS_COMPILED = {
    "vav-cooling-only": {
        "name": "VAV",
        "ztemp": "Zone_Air_Temperature_Sensor",
        "occ": "Occupancy_Sensor",           # optional
        "co2": "CO2_Level_Sensor",           # optional
        "dat": "Discharge_Air_Temperature_Sensor",
        "dmp": "Damper",
        "dmp-dmppos": "Damper_Position_Command",
        "zone": "HVAC_Zone"
    },
    "vav-with-reheat": {
        "name": "VAV",
        "name-ztemp": "Zone_Air_Temperature_Sensor",
        "name-occ": "Occupancy_Sensor",           # optional
        "name-co2": "CO2_Level_Sensor",           # optional
        "name-dat": "Discharge_Air_Temperature_Sensor",
        "name-dmp": "Damper",
        "name-dmp-dmppos": "Damper_Position_Command",
        "name-zone": "HVAC_Zone",
        "rhc": "Heating_Coil",
        "rhc-cmd": "Position_Command"
    },
    "multiple-zone-ahu": {
        "name": "AHU",
        "sa_fan": "Supply_Fan",
        "sa_fan-start_stop": "Start_Stop_Command",
        "sa_fan-speed": "Frequency_Command",
        "sa_fan-status": "Fan_Status",       # optional
        "ra_damper": "Damper",
        "ra_damper-dmppos": "Damper_Position_Command",
        "clg_coil": "Cooling_Coil",
        "clg_coil-valve_cmd": "Valve_Position_Command",
        "htg_coil": "Heating_Coil",
        "htg_coil-valve_cmd": "Valve_Position_Command",
        "sa_temp": "Supply_Air_Temperature_Sensor",
        "oa_temp": "Outside_Air_Temperature_Sensor",
        "ma_temp": "Mixed_Air_Temperature_Sensor",    # optional
        "ra_temp": "Return_Air_Temperature_Sensor",   # optional
        "filter_pd": "Filter_Differential_Pressure_Sensor"  # optional
    },
    "with-relief-damper": {
        "name": "AHU",   # composed on the AHU's 'name'
        "relief-fan": "Exhaust_Fan",
        "relief-damper": "Damper",
        "relief-damper-dmppos": "Damper_Position_Command",
        "relief-damper": "Exhaust_Damper",
        "sp-sensor": "Static_Pressure_Sensor"   # optional
    },
    "with-return-fan": {
        "name": "AHU",   # composed on the AHU's 'name'
        "return-fan": "Return_Fan",
        "relief-damper": "Damper",
        "relief-damper-dmppos": "Damper_Position_Command",
        "relief-damper": "Exhaust_Damper",
        "sp-sensor": "Static_Pressure_Sensor",          # optional
        "supply-air-flow": "Supply_Air_Flow_Sensor",    # optional
        "return-air-flow": "Return_Air_Flow_Sensor"     # optional
    },
    "relief-fan": {
        "name": "Exhaust_Fan"
    },
    "return-fan": {
        "name": "Return_Fan"
    },
    "fan": {
        "name": "Fan",
        "speed": "Frequency_Command",
        "start_stop": "Start_Stop_Command",
        "status": "Fan_Status"
    },
    "damper": {
        "name": "Damper",
        "dmppos": "Damper_Position_Command"
    },
    "heating-coil": {
        "name": "Heating_Coil",
        "cmd": "Position_Command"
    },
    "sa_fan": {
        "name": "Supply_Fan",
        "start_stop": "Start_Stop_Command",
        "speed": "Frequency_Command",
        "status": "Fan_Status"   # optional
    },
    "clg_coil": {
        "name": "Cooling_Coil",
        "valve_cmd": "Valve_Position_Command"
    },
    "htg_coil": {
        "name": "Heating_Coil",
        "valve_cmd": "Valve_Position_Command"
    },
    "feeds": {
        "name": None,    # The subject of the feeds relation (type depends on context)
        "target": None   # The object of the feeds relation (type depends on context)
    }
}

In [24]:
g = Graph()
g.bind('',BLDG)

bldg_config = {'ahus': 2, 'cv': 30, 'hv': 30}
bldg_ctx_dict = {}
for i in range(bldg_config['ahus']):
    template_name = 'multiple-zone-ahu'
    ahu_name = f'ahu_{i}'
    ahu = ctx[template_name](name=ahu_name)
    ahu.bindings = {k: BLDG[f"{template_name}_{k}_{i}"] for k in ahu.parameters}
    # display(ahu.bindings)
    # also adding to condensed representation 
    for k,v in ahu.bindings.items():
        # g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{i+1}"], v))
        g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDFS[f"member"], v))
    for cv in range(bldg_config['cv']):
        template_name = 'vav-cooling-only'
        cv_name = f'cvav_{i}_{cv}'
        cvav = ctx[template_name](name=cv_name)
        cvav.bindings = {k: BLDG[f"{template_name}_{k}_{i}_{cv}"] for k in cvav.parameters}
        ctx['feeds'](name = ahu['name'], target = cvav['name'])
        for k,v in cvav.bindings.items():
            # g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{cv+1}"], v))
            g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDFS[f"member"], v))
        # print(cvav.bindings)
    for hv in range(bldg_config['hv']):
        template_name = 'vav-with-reheat'
        hv_name = f'hvav_{i}_{hv}'
        hvav = ctx[template_name](name=hv_name)
        hvav.bindings = {k: BLDG[f"{template_name}_{k}_{i}_{hv}"] for k in hvav.parameters}
        ctx['feeds'](name = ahu['name'], target = hvav['name'])
        for k,v in hvav.bindings.items():
            # g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{hv+1}"], v))
            g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDFS[f"member"], v))

wrapper_dict = get_wrapper_dict(ctx)
template_types = wrapper_dict.keys()

skip_templates = ['feeds']
for tp in template_types:
    if tp in skip_templates:
        continue
    for wrapper in wrapper_dict[tp]:
        params = wrapper.parameters
        eval_dict = {}
        for param in params:
            # if param not in TEMPLATE_PARAMS_COMPILED[tp].keys():
            #     print(param)
            # g.add((BLDG[f"{tp}-{param.replace('name-','')}"], A, BRICK[TEMPLATE_PARAMS_COMPILED[tp][param]]))
            name = f"{tp}-{param.replace('name-','')}"
            g.add((BLDG[name], A, RDF.Seq))
            eval_dict[param] = BLDG[name]
        template_graph = wrapper.template.evaluate(eval_dict)
        if not isinstance(template_graph, Graph):
            print('not a graph, cant add')
        g = g + template_graph
    # adding feeds manually 
    g.add((BLDG['multiple-zone-ahu-name'], BRICK.feeds, BLDG['vav-cooling-only-name']))
    g.add((BLDG['multiple-zone-ahu-name'], BRICK.feeds, BLDG['vav-with-reheat-name']))

In [25]:
g.serialize('brick-example-groups.ttl', format = 'ttl')

<Graph identifier=N32366f5b0b544560af0e47f89ca6691e (<class 'rdflib.graph.Graph'>)>

In [26]:
g_no_data = g.query("""
            CONSTRUCT {
                   ?s ?p ?o .
                    ?s a ?type .
                    ?o a ?otype .
            }
            WHERE {
            ?s ?p ?o .
            ?s a ?type .
            ?o a ?otype .
            FILTER EXISTS {
                    ?o a rdf:Seq .
                }
            FILTER (?type != rdf:Seq) .
            
            FILTER (?otype != rdf:Seq) .
            }
                   """).graph

def get_label(s):
    name= str(s).rsplit('-')[-1] if str(s).rsplit('-')[-1] != 'name' else str(s).rsplit('-')[-2]
    return Literal(name)
for s,p,o in g_no_data.triples((None,None,None)):    
    if p == A:
        continue               
    if p == BRICK.hasPoint:
        g_no_data.add((o, A, BRICK.Point))
    g_no_data.add((s, RDFS.label, get_label(s)))
    g_no_data.add((o, RDFS.label, get_label(o)))

g_no_data.bind('bldg',BLDG)

In [27]:
g_no_data.serialize("brick-example-groups-no-data.ttl", format = 'ttl')

<Graph identifier=N156bd2d3edf54a8395a491a456d5b72c (<class 'rdflib.graph.Graph'>)>

In [16]:
bldg.add_graph(ctx.compile())
bldg.graph.serialize('brick-example.ttl', format = 'ttl')

<Graph identifier=6a808a18-6a78-4b3f-8641-4983fb0201c3 (<class 'rdflib.graph.Graph'>)>

# Querying

In [ ]:
# example query. What is the discharge air temperature of cooling only VAVs
import sys
import os
# current_dir = os.path.dirname(__file__)
# utils_path = os.path.abspath(os.path.join(current_dir, '..', 'utils'))
sys.path.insert(0, '../utils')
from utils import query_to_df



In [42]:
bldg_query = """
SELECT ?dat 
WHERE {
    ?dat a brick:Discharge_Air_Temperature_Sensor .
    ?vav a brick:VAV .
    ?vav brick:hasPoint ?dat .
    FILTER NOT EXISTS {
        ?vav brick:hasPart ?rc .
        ?rc a brick:Heating_Coil .
    }
}
"""
query_to_df(bldg_query, bldg.graph)

,dat
0,urn:example#vav-cooling-only_dat_0_4
1,urn:example#vav-cooling-only_dat_1_12
2,urn:example#vav-cooling-only_dat_0_26
3,urn:example#vav-cooling-only_dat_0_8
4,urn:example#vav-cooling-only_dat_1_5
5,urn:example#vav-cooling-only_dat_0_0
6,urn:example#vav-cooling-only_dat_1_9
7,urn:example#vav-cooling-only_dat_1_4
8,urn:example#vav-cooling-only_dat_1_14
9,urn:example#vav-cooling-only_dat_0_21


In [45]:
# to use the alternative representation, just need to query for members of hte container
# things that don't ask for topology - really fast and easy
group_query = """
SELECT ?dat 
WHERE {
    ?datC a brick:Discharge_Air_Temperature_Sensor .
    ?vav a brick:VAV .
    ?vav brick:hasPoint ?datC .
    FILTER NOT EXISTS {
        ?vav brick:hasPart ?rc .
        ?rc a brick:Heating_Coil .
    }
    ?datC rdfs:member ?dat
}
"""
query_to_df(group_query, g)

,dat
0,urn:example#vav-cooling-only_dat_1_26
1,urn:example#vav-cooling-only_dat_0_21
2,urn:example#vav-cooling-only_dat_0_2
3,urn:example#vav-cooling-only_dat_1_8
4,urn:example#vav-cooling-only_dat_0_25
5,urn:example#vav-cooling-only_dat_1_4
6,urn:example#vav-cooling-only_dat_0_18
7,urn:example#vav-cooling-only_dat_1_2
8,urn:example#vav-cooling-only_dat_1_9
9,urn:example#vav-cooling-only_dat_0_27


In [ ]:
# things that do ask for topology are still faster when using the overlayer than without 
group_bldg_query_wrong = """
SELECT ?dat ?vav ?ahu
WHERE {
    ?dat a brick:Discharge_Air_Temperature_Sensor .
    ?vav a brick:VAV .
    ?vav brick:hasPoint ?dat .
    ?ahu brick:feeds ?vav
    FILTER NOT EXISTS {
        ?vav brick:hasPart ?rc .
        ?rc a brick:Heating_Coil .
    }
}
"""

query_to_df(group_bldg_query_wrong, bldg.graph)

,dat,vav,ahu
0,urn:example#vav-cooling-only_dat_0_25,urn:example#vav-cooling-only_name_0_25,urn:example#multiple-zone-ahu_name_0
1,urn:example#vav-cooling-only_dat_0_27,urn:example#vav-cooling-only_name_0_27,urn:example#multiple-zone-ahu_name_0
2,urn:example#vav-cooling-only_dat_1_12,urn:example#vav-cooling-only_name_1_12,urn:example#multiple-zone-ahu_name_1
3,urn:example#vav-cooling-only_dat_1_13,urn:example#vav-cooling-only_name_1_13,urn:example#multiple-zone-ahu_name_1
4,urn:example#vav-cooling-only_dat_1_27,urn:example#vav-cooling-only_name_1_27,urn:example#multiple-zone-ahu_name_1
5,urn:example#vav-cooling-only_dat_1_18,urn:example#vav-cooling-only_name_1_18,urn:example#multiple-zone-ahu_name_1
6,urn:example#vav-cooling-only_dat_0_14,urn:example#vav-cooling-only_name_0_14,urn:example#multiple-zone-ahu_name_0
7,urn:example#vav-cooling-only_dat_1_21,urn:example#vav-cooling-only_name_1_21,urn:example#multiple-zone-ahu_name_1
8,urn:example#vav-cooling-only_dat_0_13,urn:example#vav-cooling-only_name_0_13,urn:example#multiple-zone-ahu_name_0
9,urn:example#vav-cooling-only_dat_0_1,urn:example#vav-cooling-only_name_0_1,urn:example#multiple-zone-ahu_name_0


In [49]:
# Routing queries through alternative representation where topology only used to filter is still faster
group_bldg_query = """
SELECT ?dat ?vav ?ahu
WHERE {
    ?vavC a brick:VAV, rdf:Seq .
    ?vavC rdfs:member ?vav .
    ?ahu brick:feeds ?vav .
    ?vav brick:hasPoint ?dat .
    ?dat a brick:Discharge_Air_Temperature_Sensor .
    FILTER NOT EXISTS {
        ?vavC brick:hasPart ?rc .
        ?rc a brick:Heating_Coil .
    }
}
"""
query_to_df(group_bldg_query, bldg.graph + g)

,dat,vav,ahu
0,urn:example#vav-cooling-only_dat_0_4,urn:example#vav-cooling-only_name_0_4,urn:example#multiple-zone-ahu_name_0
1,urn:example#vav-cooling-only_dat_1_12,urn:example#vav-cooling-only_name_1_12,urn:example#multiple-zone-ahu_name_1
2,urn:example#vav-cooling-only_dat_0_26,urn:example#vav-cooling-only_name_0_26,urn:example#multiple-zone-ahu_name_0
3,urn:example#vav-cooling-only_dat_0_8,urn:example#vav-cooling-only_name_0_8,urn:example#multiple-zone-ahu_name_0
4,urn:example#vav-cooling-only_dat_1_5,urn:example#vav-cooling-only_name_1_5,urn:example#multiple-zone-ahu_name_1
5,urn:example#vav-cooling-only_dat_0_0,urn:example#vav-cooling-only_name_0_0,urn:example#multiple-zone-ahu_name_0
6,urn:example#vav-cooling-only_dat_1_9,urn:example#vav-cooling-only_name_1_9,urn:example#multiple-zone-ahu_name_1
7,urn:example#vav-cooling-only_dat_1_4,urn:example#vav-cooling-only_name_1_4,urn:example#multiple-zone-ahu_name_1
8,urn:example#vav-cooling-only_dat_1_14,urn:example#vav-cooling-only_name_1_14,urn:example#multiple-zone-ahu_name_1
9,urn:example#vav-cooling-only_dat_0_21,urn:example#vav-cooling-only_name_0_21,urn:example#multiple-zone-ahu_name_0
